In [1]:
# Hỏi đáp trên video youtube
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.1

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate


In [4]:
# Documents
video_id = "L7Jp38FvmnM"
try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["vi"])
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)
except TranscriptsDisabled:
    print("No captions available for this video.")

print(transcript_list)

Hello anh em, chào mừng đã quay lại với M AI. Và hôm nay mình có một thứ muốn khoe anh em, đó là chứng chỉ quản trị dự án PMP của mình mới luôn. Đó thì hôm nay mình sẽ review cho anh em cái mình học PMP ra sao, cách mình thi PMP như thế nào và mình đã học đã đạt được cái chứng chỉ PMP như thế nào để anh em tham khảo và có thể sẽ cần cho các anh em nha. Let's go. [âm nhạc] Rồi như trên màn hình á thì mình sẽ chia sẻ về PMP Exam Lesson Learn. Đó mình vừa mới học và thi PMP xong nên là mình sẽ chia sẻ toàn bộ cho anh em từ đầu đến cuối, từ lúc học đến lúc thi rồi đến lúc báo kết quả ra làm sao để anh em biết. Thì mọi người biết rồi, PMP là một cái chứng chỉ dành cho PM. Tuy nhiên thì một số người cũng hỏi mình là có làm PM đâu học PMP làm gì thì mình nói luôn là trong PMP là design là về project manager nhưng mà sau quá trình mình học thì mình thấy rằng nó có những yếu tố về giao tiếp communication về con người pup và business đó về kinh doanh nên là chúng ta học cái này sẽ có một cái nhì

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [ ]:
chunks

In [7]:
# Nhét vào vector DB
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") #<-- embeding LLM
vector_store = FAISS.from_documents(chunks, embeddings)


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever.invoke('cách đi thi PMP')

In [9]:

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)


In [10]:
question          = "Chi phí học tại PMA là bao nhiêu?"
retrieved_docs    = retriever.invoke(question)

context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)


final_prompt = prompt.invoke({"context": context_text, "question": question})

answer = llm.invoke(final_prompt)
print(answer.content)

Giá gốc của khóa học PMA là 12,9 triệu, chưa bao gồm phí thi chứng chỉ.  Có mã giảm giá được cung cấp trong video.
